In [1]:
#Loading raw dataset
import pandas as pd
df = pd.read_csv('../data/raw/supermarket_sales.csv')

Raw Profile

- shape (rows, cols)
- column list
- dtypes
- null counts per column (sorted)
- duplicate count (full-row duplicates + duplicate Invoice_ID if exists)
- unique values for categorical columns (or at least nunique + top counts)
- min/max for numeric columns

#1  Dataset Shape

In [2]:
df.shape

(1000, 17)

#2 Column Names

In [3]:
df.columns

Index(['Invoice_ID', 'Branch', 'City', 'Customer type', 'Gender',
       'Product line', 'Unit_price', 'Quantity', 'Tax 5%', 'Total', 'Date',
       'Time', 'Payment', 'cogs', 'gross margin percentage', 'gross income',
       'Rating'],
      dtype='object')

#3 Data Types

In [4]:
df.dtypes

Invoice_ID                  object
Branch                      object
City                        object
Customer type               object
Gender                      object
Product line                object
Unit_price                 float64
Quantity                     int64
Tax 5%                     float64
Total                      float64
Date                        object
Time                        object
Payment                     object
cogs                       float64
gross margin percentage    float64
gross income               float64
Rating                     float64
dtype: object

#4 Null Analysis

In [5]:
# Null counts per column (sorted)
df.isnull().sum().sort_values(ascending=False)

Gender                     2
Invoice_ID                 0
Branch                     0
City                       0
Customer type              0
Product line               0
Unit_price                 0
Quantity                   0
Tax 5%                     0
Total                      0
Date                       0
Time                       0
Payment                    0
cogs                       0
gross margin percentage    0
gross income               0
Rating                     0
dtype: int64

#5 Duplicate Analysis

In [6]:
# Invoice_ID duplicate count

df.Invoice_ID.duplicated().sum()

np.int64(0)

In [7]:
# Full-row duplicates count

df.duplicated().sum()

np.int64(0)

#6 Categorical Overview

In [8]:
# Show categorical columns
df.select_dtypes(include='object').head()

,Invoice_ID,Branch,City,Customer type,Gender,Product line,Date,Time,Payment
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,1/5/2019,13:08,Ewallet
1,226-31-3081,C,Naypyitaw,Normal,NaN,Electronic accessories,3/8/2019,10:29,Cash
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,3/3/2019,13:23,Credit card
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,1/27/2019,20:33,Ewallet
4,373-73-7910,A,Yangon,Normal,NaN,Sports and travel,2/8/2019,10:37,Ewallet


In this instance, Date and Time columns appear but will be later on converged into one singular column that won't be considered as a categorical, that's why they're excluded in the following cell. Invoice_ID an identifier that's why it's also excluded.

In [13]:
cat_c = df.select_dtypes(include='object').columns
for col in cat_c:
    if col not in ['Invoice_ID', 'Date', 'Time']:
        print(df[col].value_counts().head(),'\n')

Branch
A    340
B    332
C    328
Name: count, dtype: int64 

City
Yangon       340
Mandalay     332
Naypyitaw    328
Name: count, dtype: int64 

Customer type
Member    501
Normal    499
Name: count, dtype: int64 

Gender
Female    500
Male      498
Name: count, dtype: int64 

Product line
Fashion accessories       178
Food and beverages        174
Electronic accessories    170
Sports and travel         166
Home and lifestyle        160
Name: count, dtype: int64 

Payment
Ewallet        345
Cash           344
Credit card    311
Name: count, dtype: int64 



#7 Numeric Overview

In [9]:
df.describe()

,Unit_price,Quantity,Tax 5%,Total,cogs,gross margin percentage,gross income,Rating
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.00000
mean,55.672130,5.510000,15.379369,322.966749,307.58738,4.761905,15.379369,6.97270
std,26.494628,2.923431,11.708825,245.885335,234.17651,0.000000,11.708825,1.71858
min,10.080000,1.000000,0.508500,10.678500,10.17000,4.761905,0.508500,4.00000
25%,32.875000,3.000000,5.924875,124.422375,118.49750,4.761905,5.924875,5.50000
50%,55.230000,5.000000,12.088000,253.848000,241.76000,4.761905,12.088000,7.00000
75%,77.935000,8.000000,22.445250,471.350250,448.90500,4.761905,22.445250,8.50000
max,99.960000,10.000000,49.650000,1042.650000,993.00000,4.761905,49.650000,10.00000


'Tax 5%', 'gross margin percentage' and 'gross income' columns will be removed later on, they shouldn't be considered

#8 Date/Time Raw Inspection

In [10]:
# Date & Time are separated columns. Identify dtype and eg values

df[['Date','Time']].dtypes

Date    object
Time    object
dtype: object

In [11]:
df[['Date','Time']].head()

,Date,Time
0,1/5/2019,13:08
1,3/8/2019,10:29
2,3/3/2019,13:23
3,1/27/2019,20:33
4,2/8/2019,10:37


Raw columns used → Clean columns produced

- Invoice_ID		str		        Unique invoice identifier
- Branch			str		        {A,B or C}
- City			    str		        {Mandalay, Naypyitaw or Yangon}	
- Customer_type		str	            {Member or Normal}
- Gender		    str		        {Male, Female or Unknown} Unknown represents missing values
- Product_line		str		        {Electronic accessories, Fashion accessories, Food and beverages, Health and beauty,Home and lifestyle or Sports and travel}
- Unit_price		float		    price per unit
- Quantity		    int		        amount of products purchased by customer
- Total			    float		    invoice total as provided in raw dataset
- Datetime		    datetime[ns]	“yyyy-mm-dd hh:mm:ss” One single column combining previous columns -> Date + Time
- Weekday		    str		        Monday - Sunday
- Payment		    str		        {Cash, Credit card, Ewallet}
- cogs			    float		    Cost of Goods = Unit_price x Quantity
- Gross_income		float		    Total - cogs
- Rating			float		    0 <= x <= 10
